In [1]:
import torch

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('dataset.csv')

In [4]:
df['text']

0        Сериал очень люблю, но Академия и Земля вызыва...
1        думал, что будет лучше идея очень интересна - ...
2        с творчеством Головачева я познакомился посред...
3        то-то я и в большое неудовольствие прочитал "А...
4        как мне показалось местами сильно смахивает на...
                               ...                        
19995    Хорошо! Оставляет место для разгадывания загад...
19996    Сериал для тех кто в поиске "квики" или "секса...
19997    Я вообще-то детективы и боевики люблю, а мелод...
19998    Изменить свою жизнь с помощью джакузи.. )) не ...
19999    шимпанзе играет в пакмэна!!! и все, больше в э...
Name: text, Length: 20000, dtype: object

In [5]:
import torch.nn as nn
from tqdm import tqdm, tqdm_notebook

In [6]:
from transformers import BertTokenizer

In [7]:
from transformers import BertTokenizerFast

In [8]:
from transformers import BertConfig

In [9]:
from transformers import BertPreTrainedModel

In [10]:
from transformers import BertForPreTraining

In [11]:
from transformers import BertForMaskedLM

In [12]:
import os
RUBERT_PATH = 'C:\\Users\Aiym Sergazina\\rubert_cased_L-12_H-768_A-12_pt'
modelpath = os.path.join(RUBERT_PATH,'pytorch_model.bin')

In [13]:
os.path.isfile(os.path.join(RUBERT_PATH,'pytorch_model.bin'))

True

In [31]:
tokenizer = BertTokenizerFast.from_pretrained(RUBERT_PATH, do_lower_case=False)
config = BertConfig.from_json_file(os.path.join(RUBERT_PATH,'bert_config.json'))
model = BertForPreTraining.from_pretrained(modelpath, config=config)

In [15]:
def exp_vec(s):
    tokenized_text = tokenizer.tokenize(s)
    tokenized_text = tokenized_text[:tokenizer.max_model_input_sizes['bert-base-uncased']-2]
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    
    segments_ids = [1] * len(tokenized_text)    
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    predictions = model(tokens_tensor, token_type_ids=segments_tensors)
    sr, sd, td = predictions[0].shape
    
    vector = []
    
    for i in range(sd):
        currentArr = predictions[0][0][i].detach().numpy()
        if len(vector) == 0:
            vector = currentArr
        else:
            vector = np.add(vector, currentArr)
    return np.mean(vector)

In [16]:
os.path.isfile('Exp_as_vector.csv')

True

In [20]:
dfs = pd.read_csv('Exp_as_vector.csv', dtype='float64')
vector_means = [exp_vec(s) for s in df['text'].tolist()]
dfs['vector'] = vector_means
dfs['tonality'] = df['rating']
dfs.head()

,tonality,vector
0,6,-204.517487
1,7,-111.591545
2,10,-311.379852
3,5,-170.935226
4,6,-244.520889


In [21]:
X = np.array(dfs['tonality'])
y = np.array(dfs['vector']).reshape(-1, 1)

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
from sklearn import metrics
X_train, X_test, y_train, y_test  = train_test_split(y, X, test_size = 0.3)

In [23]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
regressor = RandomForestClassifier(n_estimators=2000, random_state=0)
R = regressor.fit(X_train, y_train)
y_predict = regressor.predict(X_test)

In [34]:
results = confusion_matrix(y_test, y_predict) 
print('Confusion Matrix :')
print(results)
print('Report : ')
print(classification_report(y_test, y_predict))

Confusion Matrix :
[[  3   2   3   6   4   2  12  13  27  21]
 [  1   6   1   4   8   4  15  21  23  28]
 [  4   3   4   9  14  10  14  34  43  44]
 [  5   4   9   8  19  11  23  39  68  68]
 [  2   4  13  15  24  26  49  68  82  99]
 [  3   7  12  16  19  21  30  71  64  78]
 [ 12  10  18  19  43  33  60 103 125 137]
 [ 13  17  24  35  70  58 113 181 252 281]
 [ 28  24  50  61  72  69 131 233 348 414]
 [ 42  27  32  73  92  71 163 245 357 524]]
Report : 
              precision    recall  f1-score   support

           1       0.03      0.03      0.03        93
           2       0.06      0.05      0.06       111
           3       0.02      0.02      0.02       179
           4       0.03      0.03      0.03       254
           5       0.07      0.06      0.06       382
           6       0.07      0.07      0.07       321
           7       0.10      0.11      0.10       560
           8       0.18      0.17      0.18      1044
           9       0.25      0.24      0.25      1430

In [35]:
def accuracy(predict, test):
    errors = abs(predict - test)
    accuracy = 100 - np.mean(100 * (errors / test))
    return accuracy

In [36]:
accuracy(y_predict, y_test)

52.63564814814815